In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize, simplex_proj

from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline

Loading BokehJS ...

In [2]:
# Simulation Params
test_name = 'Fibonacci_agent'
obs_steps = 300 # Observation steps, number of candles required by the agent for calculations
period = 120 # Observation period in minutes, also trading frequency
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_ZEC", "USDT_DASH"] # Universe
fiat_symbol = 'USDT' # Quote symbol
# Initial portfolio
init_funds = make_balance(crypto=1 / len(pairs), fiat=0.0, pairs=pairs)
# init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs) # Full fiat
data_dir = './data' # Data directory for offline testing

In [3]:
## Environment setup
# Data feed setup
tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs=pairs, balance=init_funds, load_dir=data_dir)

# Download new data from the exchange
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=300)))

# And save it to disk, if you want to
# tapi.save_data(data_dir + '/train')

# Or load data from disk
# tapi.load_data('/train')

# Environment setup
env = BacktestEnvironment(period, obs_steps, tapi, fiat_symbol, test_name)
obs = env.reset()

# Agent setup
agent = apriori.HarmonicTrader(activation=simplex_proj)

2400 intervals, or 200 days of data at 120 minutes period downloaded.


In [4]:
# Training run
# Optimization params
nb_steps = 100
batch_size = 1
nb_max_episode_steps = 66

# Params search space
search_space = {
                'err_allowed':[1e-8, 1e-1],
                'peak_order':[1, 20],
                'alpha_up':[1e-1, 1],
                'alpha_down':[1e-1, 1],
                'decay':[0.9, 1.0]
                }

# Optimization session, this may take some time
params, info = agent.fit(env, nb_steps, batch_size, search_space, nb_max_episode_steps=nb_max_episode_steps, verbose=True)
print("\n",params,"\n", env.status)

# Run on training data
agent.test(env, verbose=True)
# Display results
env.plot_results();

Optimizing model for 100 steps with batch size 1...
Optimizing benchmark...
Benchmark optimization step 8610/10000, step reward: -0.0027726108705437014
Optimum benchmark reward: 0.110186
Best Constant Rebalance portfolio found in 8614 optimization rounds:
 [0.08173497 0.22653094 0.1978787  0.04121457 0.22559949 0.01657352
 0.12896661 0.08106013 0.00044108]

Optimizing model...
Optimization step 100/100, r: -0.01777316, r std: 0.00000000, mean r: -0.04042611 ETC: 0 days 00:00:00                           
 {'peak_order': 2.8162428191047377, 'err_allowed': 0.07073661586078112, 'alpha_up': 0.5951675923460489, 'decay': 0.9487518578415607, 'alpha_down': 0.16888650682179723} 
 {'OOD': False, 'ValueError': False, 'Error': False, 'ActionError': False, 'NotEnoughFiat': False}
>> run 1/1, step 2098/2098, 100 % done, Cum r: -1.84664020, ETC: 0 days 00:00:00, Samples/s: 2.8853                             
Reward mean: -1.84664020, Reward std: 0.00000000

################### > Portfolio Performance

In [5]:
# Validation run
# Download data
tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))

# or load from disk
# env.tapi.load_data('/eval')

# Run evaluation
agent.test(env, verbose=True)
# Show results
env.plot_results();

600 intervals, or 50 days of data at 120 minutes period downloaded.
>> run 1/1, step 118/2098, 5 % done, Cum r: -0.11064866, ETC: 0 days 00:09:39.534774, Samples/s: 3.4149                       
Keyboard Interrupt: Stoping backtest
Elapsed steps: 118/2400, 4 % done.

################### > Portfolio Performance Analysis < ###################

Portfolio excess Sharpe:                 -0.121953
Portfolio / Benchmark Sharpe ratio:      1.105100 / 1.818948
Portfolio / Benchmark Omega ratio:       1.250114 / 1.428282
Portfolio / Benchmark max drawdown:      -0.246623 / -0.215211


In [ ]:
# Test run
# Download data
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=50)))
# Or load form disk
# env.tapi.load_data('/test')
# Run test
agent.test(env, verbose=True)
# Show results
env.plot_results();